# Chinese Name OCR based on CRNN

In [1]:
from keras.layers.convolutional import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Reshape,Masking,Lambda,Permute
from keras.layers import Input,Dense,Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.layers.recurrent import GRU,LSTM
from keras.layers.wrappers import Bidirectional
from keras.models import Model
from keras import backend as K
from keras.preprocessing import image
from keras.optimizers import Adam,SGD,Adadelta
from keras import losses
from keras.layers.wrappers import TimeDistributed
from keras.callbacks import EarlyStopping,ModelCheckpoint,TensorBoard
from keras.utils import plot_model
from matplotlib import pyplot as plt

import numpy as np 
import os
from PIL import Image,ImageDraw,ImageFont 
import json
import threading
import pandas as pd
from opencc import OpenCC 

import tensorflow as tf  
import keras.backend.tensorflow_backend as K  


/home/fiona/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
name_corpus = pd.read_csv("./input/chinese_names_big5.csv")

In [3]:
name_corpus.columns=['name_sim_chi','len','name_tra_chi']
name_corpus.to_csv("./input/chinese_names_big5.csv",index=False)
name_corpus.drop_duplicates(inplace=True)

In total, there are 1,203,132 Chinese names with maximum name length of 3. There are 2,361 distinct characters.

In [4]:
print("total name is {}".format(len(name_corpus)))

print("possible name length is ")
name_corpus['len'].unique()

total name is 1203132
possible name length is 


array([2, 3])

In [5]:
name_set=list(set(name_corpus['name_tra_chi']))

char_list=[]

for name in name_set:
    char_list.extend(list(name))

char_df = pd.DataFrame(data=char_list)
char_df.columns=['char']
char_df['count']=1
char_stat = char_df.groupby('char').sum().sort_values(by='count',ascending=False)
char_stat

,count
char,
王,55563
李,53945
張,51714
陳,45452
劉,43730
文,36625
林,34617
華,31981
楊,30116


In [6]:
maxlabellength = 3
img_h = 32
img_w = 248
nclass = len(char_stat)
rnnunit=256
batch_size =64

In [7]:
sample_name=name_corpus.sample(n=100)
sample_name

,name_sim_chi,len,name_tra_chi
1144538,忠霖,2,忠霖
993766,姚金贵,3,姚金貴
544610,罗华容,3,羅華容
1185141,朱晓力,3,朱曉力
253138,何丽香,3,何麗香
745186,汤亚文,3,湯亞文
1019215,於珊珊,3,於珊珊
25248,蔡旭君,3,蔡旭君
807229,王善桂,3,王善桂
882445,习晓鹏,3,習曉鵬


In [8]:
font=ImageFont.truetype('/usr/share/fonts/truetype/windows/mingliu0.ttf',24) 

In [9]:
for index, row in sample_name.iterrows():

    img = img = Image.new('L',(img_w,img_h),(255))
    draw = ImageDraw.Draw(img)  
    name = row['name_tra_chi']
    label = ""
    for chr in name:
        label = label + chr +" "
    draw.text((0,5),label.strip() ,fill=(0),font=font)  
    img.save("./train/train_"+str(index)+'.png')